In [4]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from torch.utils.data import Dataset
# 디버깅을 위한 코드
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00


In [ ]:
final_dataframe = pd.read_csv('/content/drive/MyDrive/SW프로젝트/DATA/final_dataframe.csv', encoding='utf-8')

In [ ]:
final_dataframe.head(6)

,Unnamed: 0,movieId,title,genres,userId,timestamp,tagId,relevance,tag
0,5196860,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,1,2004-09-10 03:06:38,945,0.99225,space travel
1,5132278,919,"Wizard of Oz, The (1939)",Adventure|Children|Fantasy|Musical,1,2004-09-10 03:07:01,1112,0.98275,witch
2,11297442,2683,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy,1,2004-09-10 03:07:30,958,0.99100,spoof
3,8191973,1584,Contact (1997),Drama|Sci-Fi,1,2004-09-10 03:07:36,945,0.99050,space travel
4,5647471,1079,"Fish Called Wanda, A (1988)",Comedy|Crime,1,2004-09-10 03:07:45,758,0.99700,oscar (best supporting actor)
5,4329105,653,Dragonheart (1996),Action|Adventure|Fantasy,1,2004-09-10 03:08:11,322,0.99700,dragons


In [ ]:
user_ids = final_dataframe['userId'].nunique()
print(f"userId 개수: {user_ids}")

userId 개수: 138493


In [ ]:
# 데이터프레임에서 처음 나오는 5000개의 userID만 선택
unique_user_ids = final_dataframe['userId'].unique()[:5000]
filtered_dataframe = final_dataframe[final_dataframe['userId'].isin(unique_user_ids)]
filtered_dataframe.shape
df = filtered_dataframe

- 전체 유저 수 138493명 중 5000명 사용 -> 유저 ID 사용률 27.7%  
- 전체 데이터 19800407개 중 744401개 사용 -> 데이터 사용률 26.6%
- 코드 편의를 위해 df로 데이터프레임명 변경

## 1. 데이터 시퀀스 형태로 변환하기  
1-1. 문자형 타입 장르 -> 정수형으로 변환

### 1-1. 문자형 타입 장르 -> 정수형으로 변환

In [ ]:
# user_ids = df['userId'].unique() -> 이미 정수형으로 되어있음
# movie_ids = df['MovieId'].unique() -> 이미 정수형으로 되어있음
genres_ids = df['genres'].unique()
# tag_ids = df['tag'].unique() -> tag대신 tagId 사용할거면 이미 정수형으로 되어있음

num_genres = len(genres_ids)
# num_batch = num_user // batch_size

# genre re-indexing (1~num_genres), num_genres+1: mask idx
genres2idx = pd.Series(data=np.arange(len(genres_ids))+1,index=genres_ids)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'genres': genres_ids, 'genres_idx':genres2idx[genres_ids].values}), on = 'genres', how = 'inner')
df.sort_values(['userId', 'timestamp'], inplace = True)

In [ ]:
df.head(2)

,Unnamed: 0,movieId,title,genres,userId,timestamp,tagId,relevance,tag,genres_idx
0,5196860,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,1,2004-09-10 03:06:38,945,0.99225,space travel,1
3237,5132278,919,"Wizard of Oz, The (1939)",Adventure|Children|Fantasy|Musical,1,2004-09-10 03:07:01,1112,0.98275,witch,2


In [ ]:
# 필요한 데이터만 추출
df = df[['movieId','genres_idx','userId','tagId']]
df.head(2)

,movieId,genres_idx,userId,tagId
0,924,1,1,945
3237,919,2,1,1112


In [ ]:
df.dtypes

movieId       int64
genres_idx    int64
userId        int64
tagId         int64
dtype: object

### 1-2. train, valid set 생성

In [ ]:
from collections import defaultdict
import pandas as pd

user_train = {}
user_valid = {}
user_test = {}
usernum = 0
itemnum = 0

# filtered_dataframe를 반복하면서 User 딕셔너리를 구성
for index, row in df.iterrows():
    u = row['userId']
    i = row['movieId']
    genres = row['genres_idx']
    tag = row['tagId']
    usernum = max(u, usernum)
    itemnum = max(i, itemnum)
    if u not in user_train:
        user_train[u] = {'movies': [], 'genres': [], 'tags': []}
    user_train[u]['movies'].append(i)
    user_train[u]['genres'].append(genres)
    user_train[u]['tags'].append(tag)

# User 딕셔너리를 기반으로 user_train, user_valid, user_test를 생성
for user in user_train:
    nfeedback = len(user_train[user]['movies'])
    if nfeedback < 3:
        user_valid[user] = {'movies': [], 'genres': [], 'tags': []}
        user_test[user] = {'movies': [], 'genres': [], 'tags': []}
    else:
        user_valid[user] = {
            'movies': [user_train[user]['movies'][-2]],
            'genres': [user_train[user]['genres'][-2]],
            'tags': [user_train[user]['tags'][-2]]
        }
        user_test[user] = {
            'movies': [user_train[user]['movies'][-1]],
            'genres': [user_train[user]['genres'][-1]],
            'tags': [user_train[user]['tags'][-1]]
        }

    user_train[user] = {
        'movies': user_train[user]['movies'][:-2],
        'genres': user_train[user]['genres'][:-2],
        'tags': user_train[user]['tags'][:-2]
    }

# 결과 반환
result = [user_train, user_valid, user_test, usernum, itemnum]

# user_train 예시 출력
print("User Train Data Example:")
for user, data in user_train.items():
    print(f"User {user}:")
    print(f"Movies: {data['movies']}")
    print(f"Genres: {data['genres']}")
    print(f"Tags: {data['tags']}")
    print()

    # 처음 5개 사용자의 정보만 출력
    if user >= 2:
        break

# user_valid 예시 출력
print("User Valid Data Example:")
for user, data in user_valid.items():
    print(f"User {user}:")
    print(f"Movies: {data['movies']}")
    print(f"Genres: {data['genres']}")
    print(f"Tags: {data['tags']}")
    print()

    # 처음 5개 사용자의 정보만 출력
    if user >= 2:
        break

# user_test 예시 출력
print("User Test Data Example:")
for user, data in user_test.items():
    print(f"User {user}:")
    print(f"Movies: {data['movies']}")
    print(f"Genres: {data['genres']}")
    print(f"Tags: {data['tags']}")
    print()

    # 처음 5개 사용자의 정보만 출력
    if user >= 2:
        break

User Train Data Example:
User 1:
Movies: [924, 919, 2683, 1584, 1079, 653, 2959, 337, 1304, 3996, 151, 112, 1374, 1246, 1370, 2291, 4306, 1214, 1278, 1219, 1258, 6242, 4720, 2716, 2648, 8507, 1997, 6502, 1348, 1321, 6755, 2288, 1215, 1994, 2118, 7001, 4105, 5679, 1261, 5146, 3476, 3081, 2664, 1350, 7387, 3932, 4226, 3265, 4878, 3030, 1200, 3000, 7482, 50, 541, 1136, 5952, 1198, 7153, 1222, 1201, 593, 1089, 29, 4993, 6807, 1193, 293, 47, 1262, 1196, 2692, 2761, 296, 7438, 8690, 318, 1217, 1208, 32, 260, 1387, 3499, 4128, 253, 4754, 1333, 2644, 2762, 1266, 1080, 1249, 1240, 4011, 2804, 1358, 2944, 1259, 2542, 1090, 4027, 2918, 1036, 2194, 8636, 6333, 6539, 1291, 589, 3037, 1243, 223, 2947, 8961, 8368, 2872, 1097, 2140, 6093, 5816, 4896, 3479, 1967, 31696, 2193, 8482, 6774, 2138, 3153, 7757, 4467, 2968, 7045, 2174, 5026, 5797, 4571, 2173, 5039, 5540, 7164, 2021, 7046, 2143, 2100, 7247, 4911, 4446, 367, 6754, 2628, 5898, 4941, 1009, 3489, 2, 1848, 7389, 4980, 4915, 5040, 3438, 1920, 5171, 

### 1-3. 필요한 정보 추출  
- 목적1 : 각 요소들 임베딩 하는 과정에서 차원 파악을 위해 (num_tags, num_genres, num_movies)
- 목적2 : max_length 잡을 때 근거 (average_length)

In [ ]:
# 데이터셋을 로드하고 필요한 정보를 추출
# 유저 딕셔너리에서 태그, 장르, 영화 아이디 추출
tags = set()
genres = set()
movies = set()
total_length = 0
num_users = len(user_train)  # 사용자 수

for user_data in user_train.values():
    tags.update(user_data['tags'])
    genres.update(user_data['genres'])
    movies.update(user_data['movies'])
    total_length += len(user_data['movies'])

# 시퀀스 길이의 평균 계산
average_length = total_length / num_users

# 고유한 값들의 수를 세어서 설정
num_tags = len(tags)
num_genres = len(genres)
num_movies = len(movies)

# 설정된 값들 출력
print("num_tags:", num_tags)
print("num_genres:", num_genres)
print("num_movies:", num_movies)
print("average_length:", average_length)

num_tags: 810
num_genres: 912
num_movies: 10052
average_length: 146.8802


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertModel
from datetime import datetime
torch.backends.cudnn.benchmark = False

In [ ]:
hidden_units = 256  # 주로 사용되는 BERT-base의 hidden_units
num_heads = 2
num_layers = 3
max_len = 100 # 평균
dropout_rate = 0.1  # 예시로 0.1로 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 가능하면 GPU를 사용, 그렇지 않으면 CPU를 사용
batch_size = 64
lr = 0.001  # 원하는 학습률 값으로 설정
mask_prob = 0.15

## 2. 모델에 필요한 구성요소 정의

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate)  # dropout rate

    def forward(self, Q, K, V, mask):
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)
        # 유사도가 0 인 곳은 -infinity 로 치환하여 softmax 결과가 0 이 되도록 함
        attn_score = attn_score.masked_fill(mask == 0, -1e9)
        # 각 sequence 에서 상대적 중요도를 나타낸다.
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # dim of output: batchSize x num_head x seqLen x hidden_units
        return output, attn_dist

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head 의 수
        self.hidden_units = hidden_units

        # query, key, value, output 생성을 위해 linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)

        # scaled dot product attention module 을 사용하여 attention 계산
        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate)
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        # layer normalization
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6)

    def forward(self, enc, mask):
        residual = enc # residual connection 을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value 를 ( num_head ) 개의 Head 로 나누어 각기 다른 linear projectoin 을 통과
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)

        # Head 별로 각기 다른 attention 이 가능하도록 Transpose 후 각각 attention 에 콩과
        Q, K, V = Q.transpose(1,2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # 다시 Transpose 한 후 모든 head 들의 attention 결과를 합친다.
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, seqlen, -1)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        # SASRec 과의 dimension 차이에 유의
        self.W_1 = nn.Linear(hidden_units, 4 * hidden_units)
        self.W_2 = nn.Linear(4 * hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        # activation: relu -> gelu
        output = self.W_2(F.gelu(self.dropout(self.W_1(x))))
        output = self.layerNorm(self.dropout(output) + residual)
        return output

In [ ]:
class BERT4RecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(BERT4RecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

## 3. 최종 모델 BERT4Rec

In [ ]:
class BERT4Rec(nn.Module):
    def __init__(self, num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device):
        super(BERT4Rec, self).__init__()

        # 기본 모델 세팅 #
        self.num_user = num_user
        self.num_item = num_item
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.max_len = max_len
        self.device = device

        # 임베딩 레이어 정의 # -> 임베딩 정의와 임베딩 레이어 정의는 다름 -> 임베딩 레이어에 input seq를 넣어서 임베딩을 얻는 것
        ## 수정 : layer의 차원의 순서가 바뀐 듯 싶어 다시 바꿔주었습니다. 블로그에서 잘못 설정한 것 같습니다. ##
        self.movie_emb_layer = nn.Embedding(num_item + 2, hidden_units, padding_idx=0)
        self.tags_emb_layer = nn.Embedding(num_item + 2,hidden_units, padding_idx=0)
        self.genres_emb_layer = nn.Embedding(num_item + 2, hidden_units, padding_idx=0)
        self.pos_emb_layer = nn.Embedding(max_len,hidden_units)
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.emb_layernorm_layer = nn.LayerNorm(hidden_units, eps=1e-6)

        ## 레이어 개수 만큼 BERT4RecBlock 쌓음 (각 레이어마다 head는 num_heads 개수 만큼 존재)
        self.blocks = nn.ModuleList([BERT4RecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])
        self.out = nn.Linear(self.hidden_units, self.num_item + 1)

    # 모델의 forward 메서드 수정
    def forward(self, movie_seq, genre_seq, tag_seq):
        movie_emb = self.movie_emb_layer(movie_seq)
        tag_emb = self.tags_emb_layer(tag_seq)
        genre_emb = self.genres_emb_layer(genre_seq)

        positions = torch.arange(movie_seq.shape[1]).unsqueeze(0).repeat(movie_seq.shape[0], 1).to(self.device)
        position_emb = self.pos_emb_layer(positions)

        seqs = movie_emb + tag_emb + genre_emb + position_emb
        seqs = self.emb_layernorm_layer(self.dropout_layer(seqs))

        mask = (movie_seq > 0).unsqueeze(1).repeat(1, movie_seq.shape[1], 1).unsqueeze(1).to(self.device)

        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        out = self.out(seqs)
        return out

In [ ]:
# 모델 정의
model = BERT4Rec(num_users, num_movies, hidden_units, num_heads, num_layers, max_len, dropout_rate, device)
criterion = nn.CrossEntropyLoss(ignore_index=0) # label 이 0 인 경우 무시
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from tqdm import tqdm

## 4. 데이터 정의 및 모델 실행

In [ ]:
class SeqDataset(Dataset):
    def __init__(self, user_train, num_user, num_item, max_len, mask_prob,model):
        self.user_train = user_train
        self.num_user = num_user
        self.num_item = num_item
        self.max_len = max_len
        self.mask_prob = mask_prob
        self.model = model

    def __len__(self):
    	# 총 user 의 수 = 학습에 사용할 sequence 의 수
      return self.num_user

    def __getitem__(self,user):
    	# iterator 를 구동할 때 사용
      user_train = self.user_train[user]
      movie_seq = self.user_train[user]['movies'](torch.LongTensor(movie_tokens).to(self.device))
      tag_seq = self.user_train[user]['tags'](torch.LongTensor(tag_tokens).to(self.device))
      genre_seq = self.user_train[user]['genres'](torch.LongTensor(genre_tokens).to(self.device))

      movie_emb = self.movie_emb_layer(movie_seq)  # Assuming movie_emb is calculated from movie_seq
      tag_emb = self.tags_emb_layer(tag_seq)  # Assuming tag_emb is calculated from tag_seq
      genre_emb = self.genres_emb_layer(genre_seq)  # Assuming genre_emb is calculated from genre_seq
      positions = torch.arange(movie_seq.shape[1]).unsqueeze(0).repeat(movie_seq.shape[0], 1).to(self.device)
      position_emb = self.pos_emb_layer(positions)

      seqs = movie_emb + tag_emb + genre_emb + position_emb
      seqs = self.emb_layernorm_layer(self.dropout_layer(seqs))

      movie_tokens = []
      movie_labels = []
      tag_tokens = []
      genre_tokens = []

      for m,t,g in zip(movie_seq,tag_seq, genre_seq) :
          prob = np.random.rand()
          if prob < self.mask_prob:
            prob /= self.mask_prob

            # BERT 학습
            # random 하게 80% 를 mask token 으로 변환
            if prob < 0.8 :
              # masking
                # mask_index : num_item + 1 , 0 : pad, 1 ~ num_item : item index
                movie_tokens.append(self.num_item + 1 )
            # random 하게 10% 를 random token 으로 변환
            elif prob < 0.9 :
                # item random sampling
                movie_tokens.append(np.random.randint(1, self.num_item+1))
            else :
            # 나머지 10% 를 original token 으로 사용
              movie_tokens.append(m)

            movie_labels.append(m)
          # training 에 사용하지 않음
          else:
            movie_tokens.append(m)
            movie_labels.append(0) # 학습에 사용하지 않음, trivial

          tag_tokens.append(t)
          genre_tokens.append(g)

      movie_tokens = movie_tokens[-self.max_len:]
      movie_labels = movie_labels[-self.max_len:]
      mask_len = max(0, self.max_len - len(movie_tokens))

      tag_tokens = tag_tokens[-self.max_len:]
      genre_tokens = genre_tokens[-self.max_len:]

      # zero padding
      # tokens 와 labels 가 max_len 보다 짧으면 zero padding 을 해준다.
      movie_tokens = [0] * mask_len + movie_tokens
      movie_labels = [0] * mask_len + movie_labels
      tag_tokens = [0] * mask_len + tag_tokens
      genre_tokens = [0] * mask_len + genre_tokens

      return (torch.LongTensor(movie_tokens),
              torch.LongTensor(movie_labels),
              torch.LongTensor(tag_tokens),
              torch.LongTensor(genre_tokens)
      )

In [ ]:
seq_dataset = SeqDataset(user_train, num_users, num_movies, max_len, mask_prob,model)
data_loader = DataLoader(dataset=seq_dataset, batch_size = batch_size, shuffle=True, pin_memory = True )

In [ ]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# 현재 GPU의 사용 가능한 메모리 용량과 사용 중인 메모리 양을 확인
print(torch.cuda.get_device_properties(0))
print(torch.cuda.max_memory_allocated(0))

PyTorch version: 2.1.0+cu118
CUDA available: True
CUDA version: 11.8
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)
0


In [ ]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(1, num_epochs + 1):
    tbar = tqdm(data_loader)
    for step, (movie_seqs, labels, tag_seqs, genre_seqs) in enumerate(tbar):
      model = model.to(device)
      movie_seqs = movie_seqs.to(device)
      labels = labels.to(device)
      tag_seqs = tag_seqs.to(device)
      genre_seqs = genre_seqs.to(device)

      # 데이터 전처리를 여기서 수행
      movie_emb = model.movie_emb_layer(movie_seqs)
      tag_emb = model.tags_emb_layer(tag_seqs)
      genre_emb = model.genres_emb_layer(genre_seqs)

      positions = torch.arange(movie_seqs.shape[1]).unsqueeze(0).repeat(movie_seqs.shape[0], 1).to(device)
      position_emb = model.pos_emb_layer(positions)

      seqs = movie_emb + tag_emb + genre_emb + position_emb
      seqs = model.emb_layernorm_layer(model.dropout_layer(seqs))
      # eqs = movie_emb + tag_emb + genre_emb + position_emb
      # print(movie_seqs)

      # 모델 예측 및 학습을 수행
      logits = model(seqs, tag_seqs, genre_seqs)  # 파라미터 순서에 주의
      logits = logits.view(-1, logits.size(-1))

      optimizer.zero_grad()
      loss = criterion(logits, labels.view(-1).to(device))
      loss.backward()
      optimizer.step()

      tbar.set_description(f'Epoch: {epoch:3d}| Step: {step:3d}| Train loss: {loss:.5f}')

  0%|          | 0/79 [00:00<?, ?it/s]


UnboundLocalError: ignored

In [ ]:
model.eval()

NDCG = 0.0
HIT = 0.0

num_item_sample = 100
num_user_sample = 1000
# 1000 개만 sampling 하여 evaluation
users = np.random.randint(0, num_user, num_user_sample)

for u in users:
    # 다음 아이템을 예측하기 위한 input token
    seq = (user_train[u] + [num_item + 1 ])[-max_len:]
    rated = set(user_train[u] + user_valid[u])
    item_idx = [user_valid[u][0]] + [random_neg(1, num_item + 1, rated) for _ in range(num_item_sample)]

    with torch.no_grad():
        predictions = -model(np.array([seq]))
        predictions = predictions[0][-1][item_idx] # sampling
        rank = predictions.argsort().argsort()[0].item()

    if rank < 10:
        NDCG += 1 / np.log2(rank + 2)
        HIT += 1

print(f'NDCG@10: {NDCG/num_user_sample}| HIT@10: {HIT/num_user_sample}')